<a href="https://colab.research.google.com/github/jonochurch/DAQ-files/blob/main/DH_bike_suspension_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade plotly maybe no longer needed?
!pip install statsmodels
!pip install --upgrade scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import math
from numpy import diff
from plotly.subplots import make_subplots
from google.colab import data_table
from vega_datasets import data
import scipy.interpolate
from scipy.interpolate import splev, splrep
from scipy import signal
from scipy.interpolate import UnivariateSpline
from scipy.interpolate.fitpack2 import RectBivariateSpline
from scipy import integrate

# data_table.enable_dataframe_formatter()


In [ ]:
# data_table.disable_dataframe_formatter()

In [ ]:
# # # df=pd.read_csv('/content/drive/MyDrive/The SUSPENSION LAB/Brands/GT/DH_bike_database.csv', index_col=0)
# # df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Frame Data/Trek Remedy 8.csv', header=None)
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Frame Data/frame_database.csv', index_col=(0,1),header=None)

df

2       3       4       5       6    \
0                         1                                                  
Evil V1                   position  0.0000  0.5556  1.1111  1.6667  2.2222   
                          Ratio     3.0454  3.0213  2.9979  2.9752  2.9532   
Giant Trance              position  0.0000  1.3636  2.7273  4.0909  5.4545   
                          Ratio     2.8397  2.8269  2.8143  2.8019  2.7897   
Norco Range               position  0.0000  1.7172  3.4343  5.1515  6.8687   
                          Ratio     3.2018  3.1884  3.1753  3.1623  3.1496   
Whyte e180                position  0.0000  1.7172  3.4343  5.1515  6.8687   
                          Ratio     3.2477  3.2315  3.2154  3.1996  3.1838   
Ibis Ripmo                position  0.0000  1.4848  2.9697  4.4545  5.9394   
                          Ratio     3.1782  3.1555  3.1334  3.1119  3.0909   
Yeti SB6                  position  0.0000  1.5152  3.0303  4.5455  6.0606   
                          Ratio     2.4498  2.4486  2.4475  2.4463  2.4450   
Nomad4                    position  0.0000  1.7172  3.4343  5.1515  6.8687   
                          Ratio     3.5018  3.4841  3.4666  3.4493  3.4322   
Pivot Firebird 2022       position  0.0000  1.6566  3.3131  4.9697  6.6263   
                          Ratio     3.2487  3.2268  3.2052  3.1839  3.1630   
Zerode Katipo             position  0.0000  1.6162  3.2323  4.8485  6.4646   
                          Ratio     2.8291  2.8294  2.8294  2.8290  2.8284   
Trek Rail                 position  0.0000  1.5152  3.0303  4.5455  6.0606   
                          Ratio     2.9006  2.8969  2.8930  2.8889  2.8846   
Giant Reign 29 2020       position  0.0000  1.4747  2.9495  4.4242  5.8990   
                          Ratio     2.8697  2.8559  2.8424  2.8289  2.8157   
Devinci Spartan 2019 29   position  0.0000  1.6162  3.2323  4.8485  6.4646   
                          Ratio     2.9189  2.9088  2.8987  2.8887  2.8787   
Devinci Spartan 2019 27.5 position  0.0000  1.6162  3.2323  4.8485  6.4646   
                          Ratio     3.0381  3.0265  3.0148  3.0033  2.9917   
Transition Sentinel V2 29 position  0.0000  1.5152  3.0303  4.5455  6.0606   
                          Ratio     2.7690  2.7603  2.7517  2.7430  2.7344   
Whyte S-150 180           position  0.0000  1.4646  2.9293  4.3939  5.8586   
                          Ratio     2.9993  2.9928  2.9862  2.9795  2.9727   

                                       7        8        9        10   \
0                         1                                             
Evil V1                   position  2.7778   3.3333   3.8889   4.4444   
                          Ratio     2.9319   2.9113   2.8913   2.8719   
Giant Trance              position  6.8182   8.1818   9.5455  10.9091   
                          Ratio     2.7777   2.7658   2.7541   2.7427   
Norco Range               position  8.5859  10.3030  12.0202  13.7374   
                          Ratio     3.1370   3.1246   3.1124   3.1003   
Whyte e180                position  8.5859  10.3030  12.0202  13.7374   
                          Ratio     3.1682   3.1528   3.1375   3.1223   
Ibis Ripmo                position  7.4242   8.9091  10.3939  11.8788   
                          Ratio     3.0706   3.0507   3.0315   3.0127   
Yeti SB6                  position  7.5758   9.0909  10.6061  12.1212   
                          Ratio     2.4438   2.4424   2.4411   2.4397   
Nomad4                    position  8.5859  10.3030  12.0202  13.7374   
                          Ratio     3.4154   3.3988   3.3824   3.3662   
Pivot Firebird 2022       position  8.2828   9.9394  11.5960  13.2525   
                          Ratio     3.1425   3.1224   3.1025   3.0831   
Zerode Katipo             position  8.0808   9.6970  11.3131  12.9293   
                          Ratio     2.8275   2.8262   2.8247   2.8229   
Trek Rail                 position  7.5758   9.0909  10.6061  12.1212   
         

In [ ]:
bike='Trek Rail'
points2=df.loc[(bike,'position'),:]
lr2=df.loc[(bike,'Ratio'),:]
weight=85


In [ ]:
#@title
# position=np.array(bikes['GT f19 29r Rckr VS'][7:11])
# ratio=np.array(bikes['GT f19 29r Rckr VS'][3:7])
# bikes=df.T
# position2=np.array(bikes['Trek Session idler 20%'][7:11])
# ratio2=np.array(bikes['Trek Session idler 20%'][3:7])

# position2=np.array(df[0])
# ratio2=np.array(df[1])


# position2=np.array([0,60,180,200])
# ratio2=np.array([2.95,2.85,2.41,2.32])
# position2=np.array([0,60,110,180,213])
# ratio2=np.array([3.35,3.08,2.85,2.52,2.4])

# position2=np.array([0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110,115,120,125,130,135,140,145,150,155,160,164])
# ratio2=np.array([3.14,3.128,3.113,3.096,3.078,3.058,3.038,3.017,2.995,2.973,2.951,2.928,2.907,2.885,2.863,2.842,2.82,2.8,2.779,2.759,2.74,2.722,2.703,2.686,2.669,2.653,2.639,2.624,2.611,2.6,2.589,2.579,2.572,2.566])

# position2=np.array([0,10,20,30,40,50,55]) #Evil V1 Rob Stone
# ratio2=np.array([3.05,2.7,2.55,2.50,2.45,2.4,2.35])

# position2=np.array([0,30,60,90,120,135]) #Giant Trance 
# ratio2=np.array([2.84,2.60,2.44,2.35,2.3,2.3])

# position2=np.array([0,30,60,90,120,150,170]) #Norco Range
# ratio2=np.array([3.2,3.0,2.83,2.7,2.58,2.46,2.36])

# position2=np.array([0,30,60,90,120,150,170]) #Whyte e180
# ratio2=np.array([3.25,2.98,2.77,2.6,2.46,2.37,2.34])

# position2=np.array([0,30,60,90,120,147]) ##Ibis Ripmo
# ratio2=np.array([3.18,2.82,2.65,2.59,2.55,2.52])

# position2=np.array([0,30,60,90,120,150]) ##Yeti SB6
# ratio2=np.array([2.45,2.42,2.39,2.36,2.36,2.39])

# position2=np.array([0,40,80,120,160,170]) ##Nomad4
# ratio2=np.array([3.5,3.15,2.86,2.65,2.42,2.36])

# position2=np.array([0,30,60,90,130,164]) #Pivot Firebird 2022
# ratio2=np.array([3.25,2.9,2.66,2.47,2.28,2.12])

# position2=np.array([0,40,80,120,160]) #Zerode Katipo
# ratio2=np.array([2.83,2.75,2.55,2.26,1.97])

# position2=np.array([0,30,60,90,120,146]) #Giant Reign 29 2020
# ratio2=np.array([2.87,2.62,2.44,2.32,2.27,2.28])

# position2=np.array([0,40,80,120,160]) #Devinci Spartan 2019 29"
# ratio2=np.array([2.92,2.67,2.46,2.27,2.16])

# position2=np.array([0,40,80,120,160]) #Devinci Spartan 2019 27.5"
# ratio2=np.array([3.04,2.76,2.55,2.35,2.23])

# position2=np.array([0,30,60,90,120,150]) ##Trek Rail
# ratio2=np.array([2.9,2.8,2.67,2.52,2.44,2.4])

# position2=np.array([0,30,60,90, 120,150]) #Transition Sentinel V2 29"
# ratio2=np.array([2.77,2.60,2.45,2.33,2.22,2.17])

# position2=np.array([0,30,60,90, 120,140]) #Yeti sb140
# ratio2=np.array([2.675,2.626,2.585,2.525,2.435,2.3])

# position2=np.array([0,30,60,90, 120,150]) #Yeti sb150
# ratio2=np.array([2.675,2.61,2.55,2.49,2.4,2.21])

# position2=np.array([0,40,80,120,168]) #Rocky Mountain Slayer 2020
# ratio2=np.array([3.02,2.96,2.75,2.43,1.98])

# position2=np.array([0,30,60,90,120,150,180]) #Specialized Kenevo 2020
# ratio2=np.array([3.3,3.05,2.99,2.85,2.74,2.67,2.64])

# position2=np.array([0,40,80,120,160]) #Giant Reign-E
# ratio2=np.array([3.07,2.71,2.50,2.4,2.37])

# position2=np.array([0,30,60,90,120,153]) #Kona Process 153
# ratio2=np.array([2.56,2.567,2.512,2.44,2.372,2.32])

# position2=np.array([0,30,60,90,120,150,180]) #Norco Shore
# ratio2=np.array([3.09,2.85,2.67,2.57,2.42,2.36,2.36])

# position2=np.array([0,30,60,90,120,150]) #Whyte S-150crs v2
# ratio2=np.array([3.089,2.906,2.745,2.610,2.504,2.44])

# BO=150
# position2=np.linspace(0,BO,10)
# ratio2=np.linspace(2.96,2.35,10)


# print(position2)
# print(ratio2)


In [ ]:
#@title


# travel2=position2[-1]


# points3=np.arange(0,travel2,10)
# points2=np.append(points3,travel2)

# # inter2=scipy.interpolate.interp1d(position2, ratio2, kind='quadratic', fill_value="extrapolate")
# # lr2=inter2(points2)

# # inter2=scipy.interpolate.interp1d(position2, ratio2, kind='quadratic', fill_value="extrapolate")
# # lr3=inter2(points2)

# spl = UnivariateSpline(position2, ratio2,)

# lr2= spl(points2)

# # window=11
# # poly=1
# # lr2 = (signal.savgol_filter(lr3,
# #                           window, # window size used for filtering
# #                           poly)) # order of fitted polynomial

# print(lr2)
# print(travel2)
# print(points2)


In [ ]:
fig=px.line(x=points2, y=lr2, width=1080, height=800,line_shape='spline')#, markers=True, #range_y=(1.95,3.35) 
 #)

fig.add_trace(go.Scatter(x=points2, y=lr2,
                    line_shape='spline'))
fig.update_layout(title=bike, title_font_size=25,
                   xaxis_title='travel',
                   yaxis_title='ratio'                   )

fig.show()

In [ ]:
grad2=1/lr2

In [ ]:
inc2=points2[1]
shock2=[]
pos2=0
for a in grad2:
  
  shock2.append(pos2)
  
  x=a*inc2
  pos2=pos2+x
print(shock2[-1])

# inc2=points2[1]

# shock2=[]
# pos2=0
# for a in grad2:
  
#   shock2.append(pos2)
  
#   x=inc2/a
#   pos2=pos2+x
# shock2


57.55546861008919


In [ ]:
shock2=np.array(shock2)


In [ ]:
#@title
## for converting shock position to wheel position

# temp=shock2
# shock2=points2
# points2=temp
# print(shock2)
# #print(points2)

In [ ]:
# def natural_frequency(wo, m) : #input natural frequency (rad/s) & mass, returns spring rate
#     k=(wo**2)*m
#     return k
def natural_frequency(k, m) : 
    Wo = math.sqrt(k/m)
    return Wo
def dampingrate(f, m, r):
    damping = 2 * r * f * m #natural freq in rad/s x weight x damping ratio
    return damping   

In [ ]:
shockstroke=np.ceil(shock2[-1])

target_sag=33
US=12

Travel=points2[-1]
print(Travel)



150.0


In [ ]:
rearweight=(weight+US)*.60
m =  (rearweight*9.81) #weight on wheel in N
sagratio=scipy.interpolate.interp1d(points2, lr2,  kind='quadratic')

sagpc=Travel*(target_sag/100)

lev=sagratio(sagpc)

shock_pos=scipy.interpolate.interp1d(points2, shock2,  kind='quadratic')
shock_sag=shock_pos(sagpc)
springmetric=(m*lev)/shock_sag


# wheelforce=springcurve/lr2 #Wrong shock position method!
# bottomout=np.round((wheelforce[-1]/m),2)
springrate=np.round((springmetric*5.7))

print(shock_sag/shockstroke,'%')
print(f'{weight}kg rider=',springrate, "lb/in")

# wheelrate=diff(wheelforce)/diff(points2)
# sagrate=scipy.interpolate.interp1d(points2[1:], wheelrate,  kind='quadratic')
# sag_rate=sagrate(sagpc)
# sag_freq=np.round(((natural_frequency((sag_rate*1000),rearweight))/6.28),2)
# #print(f'{target_sag}% sag=',sagpc,'mm')
# # print(imprate, "lb/in@wheel")
# # print(rate, "N/mm@wheel")
# print(f'{sag_freq}Hz')
# print(bottomout,'g')




0.3028610790674561 %
85kg rider= 503.0 lb/in


In [ ]:

#springA = np.array([450,450,450,450,450,450,450,450,450,540,540,540,540,540,540,540,540])
# springB=np.array([500,500,500,500,500,610,610,610,610,610])
size=len(lr2)
springA=np.full((size),450)
springB=np.full((size),500)
                
spring1=np.full((size),springmetric)
spring2=springA/5.7
spring3=springB/5.7
print(spring2)
print(spring3)
print(spring1)


[78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842 78.94736842
 78.94736842 78.94736842 78.94736842 78.94736842 78

In [ ]:
incs=shock2[1]
prog_spring=[]
print(incs)
ps=0
for x in spring2:
  prog_spring.append(ps)
  
  a=incs*x
  ps=ps+a

lin_spring=[]
ps=0
for x in spring3:
  lin_spring.append(ps)
  
  a=incs*x
  ps=ps+a  

rec_spring=[]
ps=0
for x in spring1:
  rec_spring.append(ps)
  
  a=incs*x
  ps=ps+a  

# prog_spring=(spring2*shock2)/5.7
# lin_spring=(spring3*shock2)/5.7
print(prog_spring)
print(lin_spring)

0.522374681100462
[0, 41.240106402668054, 82.48021280533611, 123.72031920800416, 164.96042561067222, 206.20053201334028, 247.44063841600834, 288.6807448186764, 329.92085122134444, 371.1609576240125, 412.40106402668056, 453.6411704293486, 494.8812768320167, 536.1213832346847, 577.3614896373527, 618.6015960400208, 659.8417024426889, 701.0818088453569, 742.321915248025, 783.562021650693, 824.8021280533611, 866.0422344560292, 907.2823408586972, 948.5224472613653, 989.7625536640334, 1031.0026600667013, 1072.2427664693694, 1113.4828728720374, 1154.7229792747055, 1195.9630856773736, 1237.2031920800416, 1278.4432984827097, 1319.6834048853777, 1360.9235112880458, 1402.1636176907139, 1443.403724093382, 1484.64383049605, 1525.883936898718, 1567.124043301386, 1608.3641497040542, 1649.6042561067222, 1690.8443625093903, 1732.0844689120584, 1773.3245753147264, 1814.5646817173945, 1855.8047881200625, 1897.0448945227306, 1938.2850009253987, 1979.5251073280667, 2020.7652137307348, 2062.0053201334026, 21

In [ ]:

fig = go.Figure()  
springs=(rec_spring,prog_spring,lin_spring)

#prog_int = integrate.cumulative_trapezoid(y, x, initial=0)

for s in springs:
    wheelforce=s/lr2
    bottomout=np.round((wheelforce[-1]/m),2)
    prog_int = integrate.cumulative_trapezoid(s, points2/1000, initial=0)
    wheelrate=diff(wheelforce)/diff(points2)
    force=scipy.interpolate.interp1d(wheelforce,points2,  kind='quadratic') 
    sagpoint=force(m)
    sag=np.round((sagpoint/Travel*100),1)
    shockpoint=scipy.interpolate.interp1d(points2,shock2,  kind='quadratic') 
    shocksag=shockpoint(sagpoint)
    shocksagpc=np.round((shocksag/shockstroke*100),1)
    sagrate=scipy.interpolate.interp1d(points2[1:], wheelrate,  kind='quadratic')

    rate_at_sag=np.round((sagrate(sagpoint)),2)
    sag_freq=np.round(((natural_frequency((rate_at_sag*1000),rearweight))/6.28),2)
    Dc = np.round(dampingrate(sag_freq*6.28, rearweight, 1)) 
    Dcs=np.round(Dc*(lev**2))
    bottomout=np.round((wheelforce[-1]/m),2)
    print(Dc)
    print(Dcs)
    print(rate_at_sag,'N/mm')
    print(sag_freq,'Hz')
    print(f'shocksag={shocksagpc}%')
    print(f'wheelsag={sag}%')
    print(bottomout,'g')


    fig.add_trace(go.Scatter(x=points2, y=wheelforce, line_shape='spline',mode='lines+markers'))
    fig.add_trace(go.Scatter(x=points2, y=prog_int, line_shape='spline',mode='lines'))
    fig.add_trace(go.Scatter(x=points2[1:], y=wheelrate, line_shape='spline',mode='lines'))
    fig.add_hline(y=m)               
                    
                    
fig.update_layout(width=1500, height=850)
fig.show()

 #line=dict(color='royalblue', width=4, dash='dot') 

1681.0
12380.0
12.17 N/mm
2.3 Hz
shocksag=31.1%
wheelsag=33.9%
3.33 g
1608.0
11843.0
11.12 N/mm
2.2 Hz
shocksag=34.6%
wheelsag=37.5%
2.98 g
1681.0
12380.0
12.12 N/mm
2.3 Hz
shocksag=31.3%
wheelsag=34.1%
3.31 g


In [ ]:

fig=go.Figure()
for s in springs:
    wheelforce=s/lr2
    bottomout=np.round((wheelforce[-1]/m),2)
    prog_int = integrate.cumulative_trapezoid(s, points2/1000, initial=0)
    wheelrate=diff(wheelforce)/diff(points2)
    force=scipy.interpolate.interp1d(wheelforce,points2,  kind='quadratic') 
    sagpoint=force(m)
    sag=np.round((sagpoint/Travel*100),1)
    shockpoint=scipy.interpolate.interp1d(points2,shock2,  kind='quadratic') 
    shocksag=shockpoint(sagpoint)
    shocksagpc=np.round((shocksag/shockstroke*100),1)
    sagrate=scipy.interpolate.interp1d(points2[1:], wheelrate,  kind='quadratic')

    rate_at_sag=np.round((sagrate(sagpoint)),2)
    sag_freq=np.round(((natural_frequency((rate_at_sag*1000),rearweight))/6.28),2)
    Dc = np.round(dampingrate(sag_freq*6.28, rearweight, 1)) 
    Dcs=np.round(Dc*(lev**2))
    bottomout=np.round((wheelforce[-1]/m),2)
    fig.add_trace(go.Scatter(x=points2, y=wheelforce, line_shape='spline',name='Force',mode='lines+markers'))
    fig.add_trace(go.Scatter(x=points2, y=prog_int, line_shape='spline',mode='lines', name='Work', yaxis="y2"))
    
    
    # fig.add_trace(go.Scatter(
    #     x=df["Time"],
    #     y=df["Fork Velocity"],
    #     name="Fork Velocity"
    # ))

    # fig.add_trace(go.Scatter(
    #     x=df["Time"],
    #     y=df["Front Axle"],
    #     name="Front Axle",
    #     yaxis="y2"
    # ))


    
    #Create axis objects
    fig.update_layout(
        # xaxis=dict(
        #     domain=[0.15, 0.85]
        # ),
        yaxis=dict(
            title="Force",
            titlefont=dict(
                color="#1f77b4"
            ),
            tickfont=dict(
                color="#1f77b4"
            )
        ),
        yaxis2=dict(
            title="Energy",
            titlefont=dict(
                color="#ff7f0e"
            ),
            tickfont=dict(
                color="#ff7f0e"
            ),
            #anchor="free",
            overlaying="y",
            side="right",
            #position=0.05
        ),
        
        )

    fig.update_layout(
        title_text="Forces",
        width=1200,
        height=800
    )

    fig.update_traces(hoverinfo="all", selector=dict(type='scatter'))



fig.show()